In [28]:
import os
import pandas as pd
import numpy as np
import re
from datetime import datetime,timedelta
import netCDF4 as nc
import numpy.ma as ma
import matplotlib as mpl
import matplotlib.pyplot as pltz
import xarray as xar
import rioxarray

In [31]:
file_path: str = r'../data/nwp_high_res_wind_2023092512.nc'
# file_path:str=r'E:\\05DATA\\06wind\\2023\\09\\nwp_high_res_wind_2023092512.nc'
file_path = r'E:/05DATA/06wind/2023/09/nwp_high_res_wind_2023092512.nc'


In [32]:
ds_xr=xar.open_dataset(file_path)
ds_xr

<xarray.Dataset>
Dimensions:  (lat: 501, lon: 901, time: 241)
Coordinates:
  * lat      (lat) float64 50.0 49.9 49.8 49.7 49.6 49.5 ... 0.4 0.3 0.2 0.1 0.0
  * lon      (lon) float64 90.0 90.1 90.2 90.3 90.4 ... 179.7 179.8 179.9 180.0
  * time     (time) datetime64[ns] 2023-09-25T12:00:00 ... 2023-10-05T12:00:00
Data variables:
    u10      (time, lat, lon) float32 ...
    v10      (time, lat, lon) float32 ...
    ws       (time, lat, lon) float32 ...
    wd       (time, lat, lon) float32 ...

提取time维度的所有值

In [21]:
ds_xr['time']

<xarray.DataArray 'time' (time: 241)>
array(['2023-09-25T12:00:00.000000000', '2023-09-25T13:00:00.000000000',
       '2023-09-25T14:00:00.000000000', ..., '2023-10-05T10:00:00.000000000',
       '2023-10-05T11:00:00.000000000', '2023-10-05T12:00:00.000000000'],
      dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 2023-09-25T12:00:00 ... 2023-10-05T12:00:00

In [22]:
max_lat=41.0
min_lat=16.1
max_lon=126.9
min_lon=105.0

目前温带风暴潮预报的精度为:0.1*0.1

In [23]:
mask_lon = (ds_xr.lon >= min_lon) & (ds_xr.lon <= max_lon)
mask_lat = (ds_xr.lat >= min_lat) & (ds_xr.lat <= max_lat)
cropped_ds = ds_xr.where(mask_lon & mask_lat, drop=True)

In [24]:
cropped_ds

<xarray.Dataset>
Dimensions:  (lat: 250, lon: 220, time: 241)
Coordinates:
  * lat      (lat) float64 41.0 40.9 40.8 40.7 40.6 ... 16.5 16.4 16.3 16.2 16.1
  * lon      (lon) float64 105.0 105.1 105.2 105.3 ... 126.6 126.7 126.8 126.9
  * time     (time) datetime64[ns] 2023-09-25T12:00:00 ... 2023-10-05T12:00:00
Data variables:
    u10      (time, lat, lon) float32 4.817 4.667 4.573 ... -0.07516 0.9336
    v10      (time, lat, lon) float32 -2.439 -2.514 -2.636 ... -6.25 -6.743
    ws       (time, lat, lon) float32 5.399 5.302 5.279 ... 5.503 6.319 6.808
    wd       (time, lat, lon) float32 296.9 298.3 300.0 ... 53.1 188.3 352.2

In [25]:
# 存储为新的nc文件
save_full_path: str = r'../out_data/output.nc'
cropped_ds.to_netcdf(save_full_path, format='NETCDF4', mode='w')


In [9]:
split_ds=cropped_ds.isel(time=0)

In [10]:
split_ds.rio.write_crs("epsg:4326", inplace=True)

<xarray.Dataset>
Dimensions:      (lat: 250, lon: 220)
Coordinates:
  * lat          (lat) float64 41.0 40.9 40.8 40.7 40.6 ... 16.4 16.3 16.2 16.1
  * lon          (lon) float64 105.0 105.1 105.2 105.3 ... 126.7 126.8 126.9
    time         datetime64[ns] 2023-09-18
    spatial_ref  int32 0
Data variables:
    u10          (lat, lon) float32 2.427 2.339 2.242 ... -0.04067 0.3337
    v10          (lat, lon) float32 -2.007 -1.957 -0.7257 ... 0.3036 0.1411
    ws           (lat, lon) float32 3.149 3.05 2.527 ... 0.7331 0.5726 0.5134
    wd           (lat, lon) float32 309.6 309.9 284.6 ... 153.9 191.6 248.8
Attributes:
    grid_mapping:  spatial_ref

### 此处的问题是在于coordinates 中多了time

In [15]:
# 提取 风速
split_ws_ds=xar.Dataset({'ws':split_ds['ws']})
split_ws_ds

<xarray.Dataset>
Dimensions:      (lat: 250, lon: 220)
Coordinates:
  * lat          (lat) float64 41.0 40.9 40.8 40.7 40.6 ... 16.4 16.3 16.2 16.1
  * lon          (lon) float64 105.0 105.1 105.2 105.3 ... 126.7 126.8 126.9
    time         datetime64[ns] 2023-09-18
    spatial_ref  int32 0
Data variables:
    ws           (lat, lon) float32 3.149 3.05 2.527 ... 0.7331 0.5726 0.5134

In [16]:
split_ws_ds=split_ws_ds.rio.set_spatial_dims('lat','lon')
split_ws_ds

<xarray.Dataset>
Dimensions:      (lat: 250, lon: 220)
Coordinates:
  * lat          (lat) float64 41.0 40.9 40.8 40.7 40.6 ... 16.4 16.3 16.2 16.1
  * lon          (lon) float64 105.0 105.1 105.2 105.3 ... 126.7 126.8 126.9
    time         datetime64[ns] 2023-09-18
    spatial_ref  int32 0
Data variables:
    ws           (lat, lon) float32 3.149 3.05 2.527 ... 0.7331 0.5726 0.5134

有错误:
DimensionError: y dimension not found. 'rio.set_spatial_dims()' or using 'rename()' to change the dimension name to 'y' can address this.
- 解决办法: 需要对 lat 与 lon 进行rename

In [17]:
split_ws_ds=split_ws_ds.rename_dims({'lat':'latitude','lon':'longitude'})

In [18]:
output_file=r'../out_data/output_01.tif'
split_ws_ds.rio.to_raster(output_file)

c:\Users\evase\anaconda3\envs\newoilgdal\lib\site-packages\rasterio\__init__.py:223: NotGeoreferencedWarning: The given matrix is equal to Affine.identity or its flipped counterpart. GDAL may ignore this matrix and save no geotransform without raising an error. This behavior is somewhat driver-specific.
  **kwargs)


有错误:
DimensionError: y dimension not found. 'rio.set_spatial_dims()' or using 'rename()' to change the dimension name to 'y' can address this.